In [1]:
import os
import random
import itertools
import re

# 基本的なライブラリ
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

# 描画ライブラリ
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn_analyzer import CustomPairPlot
import graphviz
import pydotplus
from IPython.display import Image
from IPython.display import HTML
from six import StringIO
from ipywidgets import interact, FloatSlider

# 前処理
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import VarianceThreshold

# 補完
from sklearn.experimental import (
    enable_iterative_imputer,
)  # IterativeImputerをimportするために必要
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

# エンコード
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder

# データセット分割
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# 特徴量選択
from sklearn.feature_selection import (
    GenericUnivariateSelect,
    f_classif,
    mutual_info_classif,
    chi2,
)

# 学習中
from tqdm import tqdm
from sklearn.model_selection import learning_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import warnings


# config python file
import config

SEED = config.SEED


from functions import *

fix_seed(SEED)


# 最大表示列数の指定（ここでは50列を指定）N
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)

%matplotlib inline

/Users/masanari/Library/CloudStorage/OneDrive-東京理科大学/projects/breast_cancer_analytics/notebooks/functions.py:278: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y: pd.Series(),


# 目的
遺伝子学的分類に基づいた、予後の2値分類を実施する。  
分類はCLAUDIN_SUBTYPEに基づいて実施。  
予後は5年、10年、15年の3つの年次に分けている。Trueで死亡であることに注意すること。

# データ読み込み
読み込み元：
    config.INTERIM_PICKLE_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/feature_selected_chi2"

サブタイプ毎のデータを使用

データの種類が多いので、辞書型で表現する  
```
+/y_dict
    +/train
        +.y5
            +.y_subtypes
        +.y10
            +.y_subtypes
        +.y15
            +.y_subtypes
    +/test
        +.y5
            +.y_subtypes
        +.y10
            +.y_subtypes
        +.y15
            +.y_subtypes
        
        
+/X_dict
    +/train
        +/X5
            +.X_subtypes
        +/X10
            +.X_subtypes
        +/X15
            +.X_subtypes
    +/test
        +/X5
            +.X_subtypes
        +/X10
            +.X_subtypes
        +/X15
            +.X_subtypes
```

In [2]:
train_dict = {}
train_dict["y5"] = pd.read_pickle(
    config.INTERIM_PICKLE_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/y5_train.pkl"
)
train_dict["y10"] = pd.read_pickle(
    config.INTERIM_PICKLE_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/y10_train.pkl"
)
train_dict["y15"] = pd.read_pickle(
    config.INTERIM_PICKLE_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/y15_train.pkl"
)


test_dict = {}
test_dict["y5"] = pd.read_pickle(
    config.INTERIM_PICKLE_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/y5_test.pkl"
)
test_dict["y10"] = pd.read_pickle(
    config.INTERIM_PICKLE_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/y10_test.pkl"
)
test_dict["y15"] = pd.read_pickle(
    config.INTERIM_PICKLE_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/y15_test.pkl"
)

y_dict = {"train": train_dict, "test": test_dict}

In [3]:
# train
X5_train_dict = {}
X10_train_dict = {}
X15_train_dict = {}

y5_train_dict = {}
y10_train_dict = {}
y15_train_dict = {}

X5_test_dict = {}
X10_test_dict = {}
X15_test_dict = {}

y5_test_dict = {}
y10_test_dict = {}
y15_test_dict = {}
dir_name = (
    config.INTERIM_PICKLE_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/feature_selected_chi2"
)

for d_name in os.listdir(dir_name):
    if d_name == "train":  # train df dict
        for f_name in tqdm(os.listdir(dir_name + "/" + d_name)):
            if re.match(r"X5_.*", f_name):
                X5_train_dict[f_name] = pd.read_pickle(
                    dir_name + "/" + d_name + "/" + f_name
                )
            elif re.match(r"X10_.*", f_name):
                X10_train_dict[f_name] = pd.read_pickle(
                    dir_name + "/" + d_name + "/" + f_name
                )
            elif re.match(r"X15_.*", f_name):
                X15_train_dict[f_name] = pd.read_pickle(
                    dir_name + "/" + d_name + "/" + f_name
                )
            elif re.match(r"y5_.*", f_name):
                y5_train_dict[f_name] = pd.read_pickle(
                    dir_name + "/" + d_name + "/" + f_name
                )
            elif re.match(r"y10_.*", f_name):
                y10_train_dict[f_name] = pd.read_pickle(
                    dir_name + "/" + d_name + "/" + f_name
                )
            elif re.match(r"y15_.*", f_name):
                y15_train_dict[f_name] = pd.read_pickle(
                    dir_name + "/" + d_name + "/" + f_name
                )
    elif d_name == "test":  # test df dict
        for f_name in tqdm(os.listdir(dir_name + "/" + d_name)):
            if re.match(r"X5_.*", f_name):
                X5_test_dict[f_name] = pd.read_pickle(
                    dir_name + "/" + d_name + "/" + f_name
                )
            elif re.match(r"X10_.*", f_name):
                X10_test_dict[f_name] = pd.read_pickle(
                    dir_name + "/" + d_name + "/" + f_name
                )
            elif re.match(r"X15_.*", f_name):
                X15_test_dict[f_name] = pd.read_pickle(
                    dir_name + "/" + d_name + "/" + f_name
                )
            elif re.match(r"y5_.*", f_name):
                y5_test_dict[f_name] = pd.read_pickle(
                    dir_name + "/" + d_name + "/" + f_name
                )
            elif re.match(r"y10_.*", f_name):
                y10_test_dict[f_name] = pd.read_pickle(
                    dir_name + "/" + d_name + "/" + f_name
                )
            elif re.match(r"y15_.*", f_name):
                y15_test_dict[f_name] = pd.read_pickle(
                    dir_name + "/" + d_name + "/" + f_name
                )

X5_train_dict = dict(sorted(X5_train_dict.items()))
X10_train_dict = dict(sorted(X10_train_dict.items()))
X15_train_dict = dict(sorted(X15_train_dict.items()))

y5_train_dict = dict(sorted(y5_train_dict.items()))
y10_train_dict = dict(sorted(y10_train_dict.items()))
y15_train_dict = dict(sorted(y15_train_dict.items()))

X5_test_dict = dict(sorted(X5_test_dict.items()))
X10_test_dict = dict(sorted(X10_test_dict.items()))
X15_test_dict = dict(sorted(X15_test_dict.items()))

y5_test_dict = dict(sorted(y5_test_dict.items()))
y10_test_dict = dict(sorted(y10_test_dict.items()))
y15_test_dict = dict(sorted(y15_test_dict.items()))

X_train_dict = {"X5": X5_train_dict, "X10": X10_train_dict, "X15": X15_train_dict}
y_train_dict = {"y5": y5_train_dict, "y10": y10_train_dict, "y15": y15_train_dict}

X_test_dict = {"X5": X5_test_dict, "X10": X10_test_dict, "X15": X15_test_dict}
y_test_dict = {"X5": y5_test_dict, "X10": y10_test_dict, "X15": y15_test_dict}

# train, testの包括
X_dict = {"train": X_train_dict, "test": X_test_dict}
y_dict = {"train": y_train_dict, "test": y_test_dict}

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 4221.86it/s]


# 各種類ベースラインの設定

In [4]:
for xt, yt in zip(X_train_dict.values(), y_train_dict.values()):
    print("-----" * 10)
    for (kx, xtc), (ky, ytc) in zip(xt.items(), yt.items()):
        assert kx.split("_")[1] == ky.split("_")[1]
        if accuracy_score(ytc, np.zeros(len(ytc))) >= 0.5:
            print('"0">"1"', ky, round(accuracy_score(ytc, np.zeros(len(ytc))), 3))
        else:
            print('"1">"0"', ky, round(accuracy_score(ytc, np.ones(len(ytc))), 3))

--------------------------------------------------
"0">"1" y5_Basal.pickle 0.655
"0">"1" y5_Her2.pickle 0.667
"0">"1" y5_LumA.pickle 0.923
"0">"1" y5_LumB.pickle 0.788
"0">"1" y5_Normal.pickle 0.821
"0">"1" y5_claudin-low.pickle 0.812
--------------------------------------------------
"0">"1" y10_Basal.pickle 0.557
"1">"0" y10_Her2.pickle 0.558
"0">"1" y10_LumA.pickle 0.783
"0">"1" y10_LumB.pickle 0.54
"0">"1" y10_Normal.pickle 0.646
"0">"1" y10_claudin-low.pickle 0.692
--------------------------------------------------
"1">"0" y15_Basal.pickle 0.577
"1">"0" y15_Her2.pickle 0.684
"0">"1" y15_LumA.pickle 0.626
"1">"0" y15_LumB.pickle 0.675
"1">"0" y15_Normal.pickle 0.571
"0">"1" y15_claudin-low.pickle 0.575


In [7]:
for xt, yt in zip(X_train_dict.values(), y_train_dict.values()):
    print("-----" * 10)
    for (kx, xtc), (ky, ytc) in zip(xt.items(), yt.items()):
        assert kx.split("_")[1] == ky.split("_")[1]
        print(kx)
        print("使用特徴量", xtc.columns)
        print("学習サンプルサイズ：", xtc.shape)
        print(ytc.value_counts())
        display(compare_bcms(xtc, ytc))

--------------------------------------------------
X5_Basal.pickle
使用特徴量 Index(['IGHG1', 'CSN3'], dtype='object')
学習サンプルサイズ： (148, 2)
0    97
1    51
Name: OS_5years, dtype: int64


11it [00:02,  4.31it/s]


,acc_train,acc_test,f1_train,f1_test
classifier,,,,
Quadratic Discriminant Analysis,0.638138,0.588095,0.512309,0.437377
Naive Bayes,0.641151,0.580952,0.511327,0.423694
Nearest Neighbors,0.729722,0.620476,0.540085,0.324220
AdaBoost,0.863377,0.585238,0.776003,0.316168
Decision Tree,0.728212,0.627619,0.562794,0.303126
Logistic Regression,0.659887,0.626667,0.313227,0.258506
Random Forest,0.682415,0.600000,0.247872,0.116667
RBF SVM,0.662883,0.619048,0.086547,0.090000
Polynomial SVM,0.657631,0.625714,0.047727,0.025000


X5_Her2.pickle
使用特徴量 Index(['GFRA1', 'U79293', 'CLCA2', 'ESR1'], dtype='object')
学習サンプルサイズ： (153, 4)
0    102
1     51
Name: OS_5years, dtype: int64


11it [00:01,  7.09it/s]


...

11it [00:00, 11.99it/s]


,acc_train,acc_test,f1_train,f1_test
classifier,,,,
Naive Bayes,0.779647,0.750000,0.803128,0.738492
Linear SVM,0.795608,0.713333,0.819952,0.709524
Logistic Regression,0.775725,0.713333,0.806814,0.709524
Polynomial SVM,0.833255,0.676667,0.845074,0.706587
Sigmoid SVM,0.571412,0.570000,0.727052,0.705830
Random Forest,0.750118,0.663333,0.803237,0.700079
RBF SVM,0.833373,0.656667,0.853165,0.685714
Quadratic Discriminant Analysis,0.793608,0.660000,0.810888,0.675079
AdaBoost,1.000000,0.680000,1.000000,0.654365


X15_claudin-low.pickle
使用特徴量 Index(['GABRP', 'C4orf7', 'SFRP1', 'ROPN1', 'SNAR-A3'], dtype='object')
学習サンプルサイズ： (80, 5)
0    46
1    34
Name: OS_15years, dtype: int64


11it [00:00, 11.06it/s]


,acc_train,acc_test,f1_train,f1_test
classifier,,,,
AdaBoost,1.000000,0.7750,1.000000,0.637143
Random Forest,0.730556,0.6875,0.688600,0.551667
Linear SVM,0.727778,0.7000,0.677498,0.540981
Naive Bayes,0.709722,0.6750,0.687103,0.537922
Logistic Regression,0.733333,0.6875,0.674204,0.509921
Quadratic Discriminant Analysis,0.727778,0.6375,0.671400,0.496537
Nearest Neighbors,0.811111,0.6750,0.769720,0.482937
RBF SVM,0.763889,0.6125,0.715524,0.456061
Decision Tree,0.758333,0.6125,0.674765,0.413333
